In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/toc3/press


In [2]:
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset

from express import features as fs
from express import labels as ls

In [3]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f8406ebfca0>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f84d811bbb0>


In [4]:
print(train_db.games().shape, test_db.games().shape)

(151, 11) (49, 11)


In [5]:
all_features = [f.__name__ for f in fs.all_features]
all_labels = [f.__name__ for f in ls.all_labels]
print("Features:", all_features)
print("Labels:", all_labels)

Features: ['actiontype', 'actiontype_onehot', 'result', 'result_onehot', 'bodypart', 'bodypart_onehot', 'time', 'startlocation', 'relative_startlocation', 'endlocation', 'relative_endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'goalscore', 'angle', 'under_pressure', 'packing_rate', 'ball_height_onehot', 'speed', 'freeze_frame_360', 'nb_opp_in_path', 'dist_opponent', 'defenders_in_3m_radius', 'closest_3_players', 'closest_11_players', 'expected_3_receiver_and_presser_by_distance']
Labels: ['concede_shots', 'counterpress']


In [6]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "train"), 
    xfns=["startlocation", "closest_11_players", "freeze_frame_360"],
    yfns=["counterpress"], 
    load_cached =False,
    nb_prev_actions = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "test"), 
    xfns=["startlocation", "closest_11_players", "freeze_frame_360"],
    yfns=["counterpress"], 
    load_cached =False,
    nb_prev_actions = 3,
)

In [7]:
train_dataset.create(train_db)
test_dataset.create(test_db)

100%|██████████| 49/49 [00:08<00:00,  5.80it/s]


In [8]:
train_dataset.features

start_x_a0  start_y_a0  start_x_a1  start_y_a1  start_x_a2  \
game_id action_id                                                               
3788741 3             41.7375      61.285     73.7625      25.415     77.0000   
        19            84.5250      59.500     41.4750      59.500     38.5875   
        27            27.2125      13.940     28.6125       9.945     32.8125   
        31            75.7750      60.690     79.1000      58.480     79.7125   
        35            79.9750      58.055     84.0875      57.290     75.7750   
...                       ...         ...         ...         ...         ...   
3943043 2283          82.0750      20.230     67.9000       0.000      4.2875   
        2286          34.3000      61.710     71.6625      67.575     13.0375   
        2302          47.7750      31.790     69.6500      55.930     66.4125   
        2304          49.6125      35.700     47.7750      31.790     47.7750   
        2309           8.6625      64.260     11.2875      65.535     12.0750   

                   start_y_a2  teammate_1_x_a0  teammate_1_y_a0  \
game_id action_id                                                 
3788741 3              24.055        53.579218        52.737630   
        19             54.400        76.496764        53.723000   
        27              0.000        21.941713        14.556432   
        31             56.185        73.332487        65.151690   
        35             60.690        78.241322        57.741576   
...                       ...              ...              ...   
3943043 2283           31.195        67.832653        16.084311   
        2286           47.345        42.269977        58.389380   
        2302           56.440        50.047145        37.296331   
        2304           31.790        49.390287        32.930291   
        2309           50.490        10.449531        59.027809   

                   teammate_1_distance_a0  teammate_2_x_a0  ...  \
game_id action_id                                           ...   
3788741 3                       14.604240        34.377131  ...   
        19                       9.890718        65.244864  ...   
        27                       5.306711        29.895983  ...   
        31                       5.086506        79.741558  ...   
        35                       1.761781        81.094272  ...   
...                                   ...              ...  ...   
3943043 2283                    14.833448        58.134253  ...   
        2286                     8.634063        41.571906  ...   
        2302                     5.956704        41.740027  ...   
        2304                     2.778608        42.628292  ...   
        2309                     5.528951        14.996899  ...   

                   opponent_9_distance_a2  opponent_10_x_a2  opponent_10_y_a2  \
game_id action_id                                                               
3788741 3                       37.652132         44.092050         51.091348   
        19                            NaN               NaN               NaN   
        27                            NaN               NaN               NaN   
        31                      24.497275               NaN               NaN   
        35                            NaN               NaN               NaN   
...                                   ...               ...               ...   
3943043 2283                          NaN               NaN               NaN   
        2286                          NaN               NaN               NaN   
        2302                          NaN               NaN               NaN   
        2304                    33.619555               NaN               NaN   
        2309                    23.482544         12.394116         25.540747   

                   opponent_10_distance_a2  opponent_11_x_a2  \
game_id action_id                                              
3788741 3                        42.589

In [9]:
test_dataset.labels

counterpress
game_id action_id              
3857255 6                 False
        17                False
        22                False
        25                False
        40                False
...                         ...
3895348 2818              False
        2834              False
        2847              False
        2848              False
        2853              False

[12406 rows x 1 columns]

In [17]:
train_dataset.labels

,counterpress
0,False
1,False
2,False
3,True
4,True
...,...
39041,False
39042,False
39043,False
39044,False


In [18]:
test_dataset.labels

,counterpress
0,False
1,False
2,False
3,False
4,False
...,...
12401,False
12402,False
12403,False
12404,False


In [19]:
train_dataset.labels["counterpress"].value_counts()

False    31605
True      7441
Name: counterpress, dtype: int64

In [20]:
test_dataset.labels["counterpress"].value_counts()

False    9954
True     2452
Name: counterpress, dtype: int64